In [3]:
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import hashlib


g = 8
seed = 10000
secret_key_1 = "123"
secret_key_2 = "456"


dataset='covertype'


original_file = '../dataset/covtype_with_key.subset.data'
origin = pd.read_csv(original_file)

np.random.seed(seed)

cover_types = origin['Cover_Type'].unique()
cover_types.sort()

def hash_mod(key, mod_value, secret_key):
    # 使用 SHA-256 生成哈希
    combined = f"{secret_key}{key}"
    hash_value = int(hashlib.sha256(combined.encode()).hexdigest(), 16)  # 将哈希转换为整数
    return hash_value % mod_value  # 对结果取模

columns_of_interest = ['Elevation', 'Aspect']  # 可根据需求更改
origin[columns_of_interest] = origin[columns_of_interest].fillna(0)  # 填充NA为0

# 提取前两位非零数字的函数
def first_two_digits(x):
    if x == 0:
        return "00"
    digits = str(x).lstrip('0.').replace('.', '')  # 去掉前导0和小数点
    if len(digits) == 1:  # 如果只有一位数字
        return digits + "0"  # 补0
    return digits[:2]  # 返回前两位


np.random.seed(seed)
shuffled_cover_types = list(cover_types)
shuffled_cover_types = sorted(shuffled_cover_types)
np.random.shuffle(shuffled_cover_types)
half_size = len(shuffled_cover_types) // 2
green_domain = shuffled_cover_types[:half_size]
red_domain = shuffled_cover_types[half_size:]
# print(green_domain, red_domain)

decimal_watermark_informations = list(range(1,129))
for decimal in decimal_watermark_informations:
    watermark_information = bin(decimal)[2:].zfill(8)  
    watermarked_data = origin.copy()
    for idx in range(len(origin)):
        selected_data = watermarked_data.loc[idx, columns_of_interest]
        first_two_digits_data = selected_data.apply(first_two_digits)
        composite_numbers = ''.join(first_two_digits_data.values)
        
        if(watermark_information[hash_mod(composite_numbers, len(watermark_information), secret_key_1)] == '1'):
            if(hash_mod(composite_numbers, g, secret_key_2) == 0):
                if watermarked_data.loc[idx, 'Cover_Type'] in red_domain:
                    perturb_value = np.random.choice(green_domain)
                    watermarked_data.loc[idx, 'Cover_Type'] = perturb_value   
         
    results = {
        'watermarked_data': watermarked_data
    }
    np.save(f"different_version_datasets/original/{dataset}-{decimal}.npy", results)